# Student name and e-mail

name : 

e-mail : 

In [4]:
USE_GOOGLE_COLAB = False
MY_GOOGLE_DRIVE_PATH = 'TP_Modelling' # where files will be stored

In [5]:
if USE_GOOGLE_COLAB:
  from google.colab import drive
  from os.path import join 
  import os
  ROOT = "/content/drive/"     # default location for the drive

  PROJECT_PATH = join(ROOT, 'MyDrive', MY_GOOGLE_DRIVE_PATH)
  BASE_PATH = join(ROOT, 'MyDrive')
  
  drive.mount(ROOT, force_remount=False)           # we mount the google drive at /content/drive

  os.chdir(BASE_PATH)

  try:
    os.mkdir(PROJECT_PATH)
  except FileExistsError:
    pass
  os.chdir(PROJECT_PATH)

  ! git clone https://github.com/GFuhr/MF_FCM6.git
  
  os.chdir('./MF_FCM6/')
  ! git pull origin master
  os.chdir('./python')

  print('current folder')
  print(os.getcwd())

# python modules

In [2]:
%load_ext cython

In [3]:

from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation

import cython
import datetime
import numpy as np
try:
    import yaml
    HAS_YAML=True
    display('Good News, YAML format accepted')
except ImportError:
    import json as js
    HAS_YAML=False
    display('only json accepted')

%matplotlib inline

'Good News, YAML format accepted'

function used to generate animations and compute computation time

# Additional functions

<font size=3>
    Functions defined here have been made to simplify representation of outputs. These functions are not part of the labs and can be used directly
</font>

In [4]:
from utils.timer import Timer
from utils.plotting import plot_results, plot_profile, figformat

In [19]:
from utils.file import save_outputs

def save_with_params(output, params):
    prefix = ''
    if params.get('C', None) is not None:
        prefix = 'diff_'
    if params.get('V', None) is not None:
        prefix += 'adv_'
    prefix += '{0:03d}_'.format(params['Nx'])
    save_outputs(output, prefix=prefix)

In [6]:
def load_from_string(params_str:str):
    """
    function used to load parameters from a string and load it into a dict used by simulations
    return : dict or ValueError if scheme is not correct
    """
    if not HAS_YAML:
        data = js.loads(params_str)
    else:
        data = yaml.safe_load (params_str)
    data['V'] = np.double(data['V'])
    try:
        data['Nx'] = np.int(data['Nx'])
    except TypeError:
        raise ValueError('Nx Value missing or not written as a number')
    try:
        data['Ny'] = np.int(data['Ny'])
    except TypeError:
        raise ValueError('Ny Value missing or not written as a number')
    except KeyError:
        print('Ny key not found')
    data['scheme'] = data['scheme'].lower().strip()
    if data['scheme'] not in ('eule', 'euli', 'rk2', 'rk4', 'cn'):
        raise ValueError('unknown time scheme')
    return data

In [7]:
# function extracting the max value and his position for each serie of a list of profiles
def extract_max(Frames):
    list_max = np.zeros(shape=(2, len(Frames)))
    idx = 0
    for frame in Frames:
        list_max[:,idx] = (frame.argmax(), frame.max())
        idx += 1
    return np.asarray(list_max)

# Definition of Finite difference functions used to resolve convection-diffusion equation

In [8]:
%%cython -f 
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
# cython: boundscheck = False
# cython: wraparound = False

import setuptools
import numpy as np
cimport cython
from cython.parallel import prange, parallel
cimport numpy as np
DTYPE = np.double
ctypedef np.double_t DTYPE_t


# definition of a matrix representation used for implicit schemes
# 
cdef class linearmatrix(object):
    cdef int size
    cdef double[:] tridiag_a
    cdef double[:]  tridiag_b
    cdef double[:]  tridiag_c
    cdef double[:]  tridiag_gam
    cdef double[:]  tridiag_bet
    cdef double[:, :] matrix
    cdef bint isinit
    cdef tuple shape

    def __cinit__(self, int size):
        self.size = size
        self.shape = (size, size)
        self.init_tridiag()

    def init_tridiag(self):
        self.isinit = True
        self.tridiag_b = np.zeros(self.size, dtype=DTYPE)
        self.tridiag_a = np.zeros(self.size-1, dtype=DTYPE)
        self.tridiag_c = np.zeros(self.size-1, dtype=DTYPE)
        self.tridiag_gam = np.zeros(self.size, dtype=DTYPE)
        self.tridiag_bet = np.zeros(self.size, dtype=DTYPE)
        self.matrix = np.zeros(self.shape, dtype = DTYPE)

    def initInvMat1D(self):
        """
        initialisation des donnees pour l'inversion d'une matrice tridiagonale
        :param a: diagonale inferieure
        :param b: diagonale principale
        :param c: diagonale superieure
        :param pGam:
        :param pBet:
        :return: None
        """
        cdef int k=0
        cdef int iSizeX = self.tridiag_b.shape[0]-1


        self.tridiag_bet[1]=self.tridiag_b[1]
        for k in range(2, iSizeX-1):
            self.tridiag_gam[k] = self.tridiag_c[k-1]/self.tridiag_bet[k-1]
            self.tridiag_bet[k] = self.tridiag_b[k]-self.tridiag_a[k]*self.tridiag_gam[k]

        for k in range(1, iSizeX-1):
            self.tridiag_bet[k] = 1./self.tridiag_bet[k]

    # routine d'inversion d'une matrice tridiagonale
    def solve(self, np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] rhs,
                 np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] inv):
        """
        routine d'inversion d'une matrice tridiagonale : A * inv = rhs
        :param rhs: vector
        :param inv: vector
        :param a:
        :param pGam:
        :param pBet:
        :return: No return
        """
        cdef long iSizeX = rhs.shape[0]
        cdef long k

        inv[1]=rhs[1]*self.tridiag_bet[1]
        for k in range(2, iSizeX-1):
            inv[k] = (rhs[k]-self.tridiag_a[k]*inv[k-1])*self.tridiag_bet[k]
        for k in range(iSizeX-2, 0, -1):
            inv[k] -= self.tridiag_gam[k+1]*inv[k+1]


    def add_diffusion(self, double C, double dx):
        if  self.isinit is False:
            self.init_tridiag()
        for k in range(0, self.size-1):
            self.tridiag_b[k] += -2.*C/dx**2
            self.tridiag_a[k] += C/dx**2
            self.tridiag_c[k] += C/dx**2
        self.tridiag_b[self.size-1] += -2.*C/dx**2

    def add_advection(self, double V, double dx):
        if self.isinit is False:
            self.init_tridiag()
        for k in range(0, self.size-1):
            self.tridiag_b[k] += -V/dx
            self.tridiag_a[k] += 0
            self.tridiag_c[k] += V/dx
        self.tridiag_b[self.size-1] += -V/dx

    def add_diag_constant(self, double C):
        for k in range(0, self.size):
            self.tridiag_b[k] += C

    def as_dense_matrix(self):
        self.matrix = np.diag(self.tridiag_b) + np.diag(self.tridiag_a, -1) \
                      + np.diag(self.tridiag_c, 1)

        
        
# function used to compute diffusion
cdef diffusion(double dx, double C, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] v, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] u):
    cdef int m = u.shape[0]-1
    cdef int i
    cdef double hx = C/dx**2

    with nogil, parallel(num_threads=4):
        for i in prange(1, m):
            u[i] += hx*(v[i+1] + v[i-1] - 2*v[i])


cdef advection(double dx, double V, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] v, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] u):
    cdef int m = u.shape[0]-1
    cdef int i, start = 1
    cdef double hx = (V/dx)
    with nogil, parallel(num_threads=4):
        for i in prange(start, m):
                u[i] += hx*(v[i]  - v[i-1])



cdef boundary(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] u):
    cdef int m = u.shape[0]-1
    # periodic
    u[0] = u[m-1]
    u[m] = u[1]
    # neumann
    # u[0] = u[2]
    # u[m] = u[m-2]
    # dirichlet
    # u[0] = -u[2]
    # u[m] = -u[m-2]
    
    

def time_step(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] pp1, **kwargs):
    cdef double dx = kwargs.get('dx')
    cdef double C = kwargs.get('C', .001)
    cdef double V = kwargs.get('V', -.001)
    diffusion(dx, C, Field_p, pp1 )
    advection(dx, V, Field_p, pp1 )
    

def eule(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] rhs, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, **kwargs):
    """
        compute explicit euler time step
        Field_p = Field_p + dt* (-V d/dx + C d2/dx2)
        params Field_p :
        params rhs : temporary array to compute RHS of equation
        params kwargs : physical parameters (C, V, dx, dt)
    """

    cdef int m = Field_p.shape[0]-1
    cdef int idx_x, start = 1

    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
            rhs[idx_x] = 0
    time_step(Field_p, rhs, **kwargs)
    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
            Field_p[idx_x] += rhs[idx_x]
    boundary(Field_p)


def euli(linearmatrix matA, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] pp1, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, **kwargs):

    cdef int m = Field_p.shape[0]
    cdef int idx_x, start = 1
    matA.solve(Field_p, pp1)
    #boundary(pp1)
    for idx_x in range(0, m):
            Field_p[idx_x] = pp1[idx_x]
    boundary(Field_p)

    
def CranckN(linearmatrix matA, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] rhs, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, **kwargs):

    eule(rhs, Field_p, **kwargs)
    euli(matA, rhs, Field_p, **kwargs)


def RK_step(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] ki, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] ppi, **kwargs):

    cdef int m = Field_p.shape[0]-1
    cdef int idx_x, start = 1
    cdef double gamma = kwargs.get('gamma', .5)
    time_step(Field_p, ki, **kwargs)
    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
                ppi[idx_x] = Field_p[idx_x] + gamma*ki[idx_x]
    boundary(ppi)


def RK4(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k1, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k2, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k3, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k4, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] y1, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] y2, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] y3, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, \
    **kwargs):

    cdef int m = Field_p.shape[0]-1
    cdef int idx_x, idx_y, start = 1

    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
                k1[idx_x] = 0
                k2[idx_x] = 0
                k3[idx_x] = 0
                k4[idx_x] = 0
                y1[idx_x] = 0
                y2[idx_x] = 0
                y3[idx_x] = 0
    kwargs['gamma'] = .5
    RK_step(Field_p, k1, y1, **kwargs)
    kwargs['gamma'] = .5
    RK_step(y1, k2, y2, **kwargs)
    kwargs['gamma'] = 1
    RK_step(y2, k3, y3, **kwargs)
    time_step(y3, k4, **kwargs)

    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
                Field_p[idx_x] = Field_p[idx_x] + \
                                    (1./6.)*(k1[idx_x]+2.*(k2[idx_x]+k3[idx_x])+k4[idx_x])
    boundary(Field_p)


def RK2(
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k1, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] k2, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] y1, \
    np.ndarray[DTYPE_t, ndim=1, negative_indices=False, mode='c'] Field_p, \
             **kwargs):

    cdef int m = Field_p.shape[0]-1
    cdef int idx_x, idx_y, start = 1

    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
                k1[idx_x] = 0
                k2[idx_x] = 0
                y1[idx_x] = 0
    kwargs['gamma'] = .5
    RK_step(Field_p, k1, y1, **kwargs)
    kwargs['gamma'] = 1
    time_step(y1, k2, **kwargs)

    with nogil, parallel(num_threads=4):
        for idx_x in prange(start, m):
                Field_p[idx_x] = Field_p[idx_x] + k2[idx_x]
    boundary(Field_p)
    
    

## function used for initial condition
If/When you modify this function, don't forget to execute again the cell (press shift+enter for that)

In [9]:
def initfield_1D(x: np.array):
    """
    generate initial profile for advdiff simulations,
    :param x: meshgrid for X values
    :return: 1D field
    """
    u0 = np.zeros(x.shape)
    u0 = np.exp(-(x-.5*x.max())**2/10)

    # exemple for gate
#     u0[:] = 1
#     u0[0:u0.shape[0]//4] = 0
#     u0[3*u0.shape[0] // 4:] = 0
    return u0

In [10]:

# run the simulation corresponding to given parameters
def simulate(verbose=False, **kwargs):
    global_params = kwargs.copy()
    
    Nx = global_params['Nx']
    dx = global_params['dx']
    dt = global_params['dt']
    Tmax = global_params['Tmax']
    Toutput = global_params['Toutput']
    C = global_params['C']
    V = global_params['V']

    # init fields and constants
    X = dx*np.linspace(0, Nx-1, num=Nx)

    Field_p_init = initfield_1D(X)

    #RK Fields
    k1 = np.zeros(Nx)
    k2 = np.zeros(Nx)
    k3 = np.zeros(Nx)
    k4 = np.zeros(Nx)
    y1 = np.zeros(Nx)
    y2 = np.zeros(Nx)
    y3 = np.zeros(Nx)

    # matrix for implicit schemes
    Mat = linearmatrix(Nx)

    # init fields and constants
    if verbose:
        display(global_params)
        display('stability analysis : ')
        display('advection : {0}'.format(np.abs(V)*dt/dx))
        display('diffusion : {0}'.format(C*dt/dx**2))

    t = 0
    tlast = 0
    global_params['C'] *= dt
    global_params['V'] *= dt


    Field_p = Field_p_init.copy()
    Frames = []
    Frames.append(Field_p_init.copy())

    # generate matrix for implicit part I + Vdu/dx - C d2u/dx2
    scheme = global_params['scheme'].lower().strip()
    if scheme == 'cn':
        global_params['V'] *= .5
        global_params['C'] *= .5
    C = global_params['C']
    V = global_params['V']
    Mat.add_advection(-V, dx)
    Mat.add_diffusion(-C, dx)
    Mat.add_diag_constant(1)
    Mat.initInvMat1D()

    iterations = 0
    with Timer() as tf:
        while(t<Tmax):
            # uncomment line depending on choosen scheme
            if scheme == 'eule':
                eule(k1, Field_p, **global_params)
            elif scheme == 'euli':
                euli(Mat, k1, Field_p, **global_params)
            elif scheme == 'rk2':
                RK2(k1, k2, y1, Field_p, **global_params)
            elif scheme == 'rk4':
                RK4(k1, k2, k3, k4, y1, y2, y3, Field_p, **global_params)
            elif scheme == 'cn':
                CranckN(Mat, k1, Field_p, **global_params)
            else:
                raise ValueError('scheme not specified : '+scheme)

            if (t - tlast) > Toutput:
                #print('processing.... {0}%'.format(int(100.*t/Tmax)))
                tlast += Toutput
                Frames.append(np.array(Field_p))
            t += dt
            iterations += 1
    if verbose:
        display('total execution time in µs : {0}'.format(tf.interval))
        display('number of snapshots : {0}'.format(len(Frames)))
        display('used time for 1 time step : {0:.2f} µs'.format(tf.interval/iterations))
    return Frames



## definition of simulation's parameters

In [11]:

# list of parameters used in both advdiff et H2D.
# Remark : the # starting line indicates a comment and is not necessary

# definition of parameter input to be used
"""
# time step
dt: .000001

# x step
dx:  .015

# y step (used only for H2D simulations)
dy: .01

# Points in X direction
Nx:  128

# Points in Y direction
Ny: 128

# modes in Y direction
Nm:  32

# wave number in Y direction
ky: .1

# end time
Tmax: .00001

# output time
Toutput: .000001

# diffusion coefficient
C:  .02

# advection coefficient
V:  -.2

# time scheme
# can be
# eule for euler explicit (default)
# euli for euler implicit
# RK2 for Runge-Kutta 2
# RK4 for Runge-Kutta 4
# CN for Cranck-Nicholson
scheme: eule
"""
;

''

# Section 0 : Introduction

Solve convection-diffusion equation : 
$$\Large \frac{\partial u(x,t)}{\partial t}+V\frac{\partial u(x,t)}{\partial x} = C\frac{\partial^2 u(x,t)}{\partial x^2}$$
$$\Large u(x, t=0) = u_0(x) $$
$$\Large u(0, t) = u(L_x, t) $$

<font size=4>
<p>First, we define parameters in a string which will then be used for the simulation. All the parameters are described in the upper cell.
This variable is converted to an usable dict through the function load_from_string.
    </p>
</font>

### Diffusion Only

$$\Large \frac{\partial u(x,t)}{\partial t} = C\frac{\partial^2 u(x,t)}{\partial x^2}$$
$$\Large u(0, t) = u(L_x, t) $$
$$\Large u(x, 0) = \exp\left( -x^2/\sigma^2\right) $$

<font size=4>
    If YAML format is not accepted, parameters should be written in the following format : each variable must have " around is name and each line terminated with a comma
</font>

In [12]:
params_ex_diff_str = """{
"C": 0.04,
"Nx": 256,
"Tmax": .000001,
"Toutput": .0000001, 
"V": 0,
"dt": 0.000000001,
"dx": 0.1,
"scheme": "eule"
}
"""
params_ex_diff = load_from_string(params_ex_diff_str)
display(params_ex_diff)

Ny key not found


{'C': 0.04,
 'Nx': 256,
 'Tmax': 1e-06,
 'Toutput': 1e-07,
 'V': 0.0,
 'dt': 1e-09,
 'dx': 0.1,
 'scheme': 'eule'}

<font size=4>
    If YAML format is accepted, parameters can be written following this format. If YAML is not recognized, you will have an error message.
</font>

In [13]:
params_ex_diff_str = """
C: 0.04
Nx: 256
Tmax: 500
Toutput: 1 
V: 0
dt: 0.12
dx: 0.1
scheme: eule
"""
params_ex_diff = load_from_string(params_ex_diff_str)
display(params_ex_diff)

Ny key not found


{'C': 0.04,
 'Nx': 256,
 'Tmax': 500,
 'Toutput': 1,
 'V': 0.0,
 'dt': 0.12,
 'dx': 0.1,
 'scheme': 'eule'}

<font size=4> 
    To run a new simulation you just need to use the function simulate with the parameters written previously as argument
</font>

In [14]:
%%time
output_ex_diff = simulate(**params_ex_diff, verbose=False)

Wall time: 55 ms


<font size=4> 
    To see the result as an animation you can use the function plot_results with the returned value from the function simulate as argument
</font>

In [15]:
%%time
plot_results(output_ex_diff)

Wall time: 31.9 s


<font size=4>
    to plot last snapshot
</font>

In [16]:
plot_profile(output_ex_diff, -1)

<font size=4>
    to plot evolution of max(u(x,t)) in time
</font>

In [17]:
figformat().apply()
plt.figure(figsize=(10,10))
plt.grid(False)
ax = plt.gca()
x = extract_max(output_ex_diff)[0]
y = extract_max(output_ex_diff)[1]
t = 1+np.arange(*x.shape)
ax.scatter(t, y, c=t)
# set log scale in y
ax.set_yscale('log')
# set log scale in x
# ax.set_xscale('log')

plt.xlabel('time')
plt.ylabel('max')
;

''

To save results of a simulation : 

In [20]:
save_with_params(output_ex_diff, params_ex_diff)

data saved in file [diff_adv_256_0000_0001.txt]


### Section 1 : diffusion

In [ ]:
# fill this parameters to resolve time evolution of 1D diffusion

params_for_diff = """
C: 
Nx: 
Tmax: 
Toutput: 
V: 0
dt: 
dx: 
scheme: eule
"""
params_diff = load_from_string(params_for_diff)
display(params_diff)

## Stability analysis

Compute and plot outputs for the following conditions :

### $C\Delta t /\Delta x^2 < 1/2$

### $C\Delta t /\Delta x^2 = 1/2$

### $C\Delta t /\Delta x^2 > 1/2$

### $C\Delta t /\Delta x^2 = 1/2 - \epsilon$

### $C\Delta t /\Delta x^2 = 1/2 + \epsilon$

<font size=4> Comparing previous results, is CFL condition confirmed numerically?</font>

### compute growth rate of diffusion in the case where CFL < 1/2

You can use the function extract_max to obtain usefull infos concerning maximal value of the profile at each output time step

<font size=4>
    Analytic growth rate for diffusion is equal to  :
<font>
$$ u(x,0) = A \sin \left( k_x x\right) \Rightarrow \gamma = -C k_x^2$$
Are the numerical and analytical values comparable?

## Runge-Kutta 4 scheme

In [ ]:
params['scheme'] = 'RK4'
params['V'] = 0

<font size=4> 
    estimate stability limit using RK4 scheme
</font>

## Implicit Euler scheme

In [ ]:
params['scheme'] = 'euli'
params['V'] = 0

Plot results for CFL = .1, .5, 1

Is implicit scheme always stable?

# Part 2 :  Advection

In [ ]:
# fill this parameters to resolve time evolution of 1D diffusion

params_for_diff = """
C: 0
Nx: 
Tmax: 
Toutput: 
V: ??
dt: 
dx: 
scheme: eule
"""
params_diff = load_from_string(params_for_diff)
display(params_diff)

kind of derivatives :

definition of advection :

simulation with $u(x,t=0) = sin$, modify the function initfield_1D

In [ ]:
def initfield_1D(x: np.array):
    """
    generate initial profile for advdiff simulations,
    :param x: meshgrid for X values
    :return: 1D field
    """
    u0 = np.zeros(x.shape)
    u0 = np.exp(-(x-.5*x.max())**2/10)

    # exemple for gate
#     u0[:] = 1
#     u0[0:u0.shape[0]//4] = 0
#     u0[3*u0.shape[0] // 4:] = 0
    return u0

simulation with $u(x,t=0) = gate$, modify the function initfield_1D

In [ ]:
def initfield_1D(x: np.array):
    """
    generate initial profile for advdiff simulations,
    :param x: meshgrid for X values
    :return: 1D field
    """
    u0 = np.zeros(x.shape)
    u0 = np.exp(-(x-.5*x.max())**2/10)

    # exemple for gate
#     u0[:] = 1
#     u0[0:u0.shape[0]//4] = 0
#     u0[3*u0.shape[0] // 4:] = 0
    return u0

Comments : 

estimation of diffusion rate

$V\Delta t/ \Delta x <1$

$V\Delta t/ \Delta x == 1$

$V\Delta t/ \Delta x >1$

$V\Delta t/ \Delta x =1-\epsilon$

$V\Delta t/ \Delta x =1+\epsilon$

estimation of stability limit using RK4 scheme

# Part 3 :  Advection+Diffusion

Estimation of diffusion rate

Estimation of diffusion rate with centered differences for convection